In [14]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_curve, auc, classification_report
import lightgbm as lgb

In [15]:
train_df = pd.read_csv("../../../dataset/open_world/openworld_train.csv")
test_df  = pd.read_csv("../../../dataset/open_world/openworld_test.csv")

In [16]:
# feature/label 분리
target_col = train_df.columns[-1]   # 마지막 컬럼이 label

# open-world label 통합
train_df[target_col] = train_df[target_col].apply(lambda x: 0 if x == 95 else 1)
test_df[target_col]  = test_df[target_col].apply(lambda x: 0 if x == 95 else 1)

X_train = train_df.drop(columns=[target_col]).values
y_train = train_df[target_col].astype(np.float32).values
X_test  = test_df.drop(columns=[target_col]).values
y_test  = test_df[target_col].astype(np.float32).values

In [17]:
# Best MLP param.
best_mlp_params = {
    "hidden_layer_sizes": (288, 80),
    "alpha": 3.813848083517318e-05,
    "learning_rate_init": 0.0017053755428803074,
    "batch_size": 128,
    "activation": "relu",
    "solver": "adam",
}

# Train MLP ensemble (seeds: 0-4)
seeds = [0, 1, 2, 3, 4]
mlp_models = []

print("\nTraining 5 MLP models...")
for s in seeds:
    mlp = MLPClassifier(
        **best_mlp_params,
        random_state=s,
        max_iter=50
    )
    mlp.fit(X_train, y_train)
    mlp_models.append(mlp)
    print(f"MLP model seed={s} done.")


Training 5 MLP models...


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=0 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=1 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=2 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=3 done.
MLP model seed=4 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
# Train LightGBM
lgb_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.03,
    "num_leaves": 63,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "min_data_in_leaf": 20,
    "max_depth": -1,
    "seed": 42,
    "num_threads": -1
}

print("\nTraining LightGBM model...")
train_dataset = lgb.Dataset(X_train, label=y_train)
lgb_model = lgb.train(
    params=lgb_params,
    train_set=train_dataset,
    num_boost_round=300
)
print("LightGBM training done.")


Training LightGBM model...
[LightGBM] [Info] Number of positive: 13300, number of negative: 7000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000717 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5234
[LightGBM] [Info] Number of data points in the train set: 20300, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.655172 -> initscore=0.641854
[LightGBM] [Info] Start training from score 0.641854
LightGBM training done.


In [19]:
# Soft Voting ensemble (MLP x5 + LGBM)
print("\nEnsembling predictions...")

proba_list = []

# MLP 5개 확률
for model in mlp_models:
    p = model.predict_proba(X_test)[:, 1]
    proba_list.append(p)

# LightGBM 확률
lgb_proba = lgb_model.predict(X_test)
proba_list.append(lgb_proba)

# 평균 확률 = soft voting
proba_ensemble = np.mean(proba_list, axis=0)
y_pred = (proba_ensemble >= 0.5).astype(int)


Ensembling predictions...


In [20]:
# Accuracy & F1
test_acc = accuracy_score(y_test, y_pred)
test_f1_macro = f1_score(y_test, y_pred, average='macro')
test_f1_micro = f1_score(y_test, y_pred, average='micro')
test_f1_weighted = f1_score(y_test, y_pred, average='weighted')

# ROC-AUC (binary)
test_roc_auc = roc_auc_score(y_test, proba_ensemble)

# PR-AUC
prec, rec, _ = precision_recall_curve(y_test, proba_ensemble)
test_pr_auc = auc(rec, prec)

print("\n========== [TEST RESULTS - MLP5 + LightGBM Ensemble] ==========")
print(f"Accuracy        : {test_acc:.4f}")
print(f"F1 (macro)      : {test_f1_macro:.4f}")
print(f"F1 (micro)      : {test_f1_micro:.4f}")
print(f"F1 (weighted)   : {test_f1_weighted:.4f}")
print(f"ROC-AUC         : {test_roc_auc:.4f}")
print(f"PR-AUC          : {test_pr_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


========== [TEST RESULTS - MLP5 + LightGBM Ensemble] ==========
Accuracy        : 0.8946
F1 (macro)      : 0.8813
F1 (micro)      : 0.8946
F1 (weighted)   : 0.8936
ROC-AUC         : 0.9570
PR-AUC          : 0.9759

Classification Report:
              precision    recall  f1-score   support

         0.0     0.8734    0.8120    0.8416      3000
         1.0     0.9046    0.9381    0.9210      5700

    accuracy                         0.8946      8700
   macro avg     0.8890    0.8750    0.8813      8700
weighted avg     0.8938    0.8946    0.8936      8700

